## 各種import

In [1]:
from io import StringIO
from pyxcp.config import readConfiguration
from pyxcp import transport
from pyxcp.transport.candriver.pc_vector import Vector # "CAN_DRIVER": "Vector"を指定する場合に必要
from pyxcp.master import Master
from collections import namedtuple
import time

## コンフィグレーションパラメータ(JSON文字列)の作成と読み込み

In [2]:
JSON = """{
"TRANSPORT": "CAN",
"CAN_DRIVER": "Vector",
"CAN_USE_DEFAULT_LISTENER": true,
"CHANNEL": "0",
"CAN_ID_MASTER": 2,
"CAN_ID_SLAVE": 1,
"CAN_ID_BROADCAST": 256,
"MAX_DLC_REQUIRED": false,
"CREATE_DAQ_TIMESTAMPS": true,
"BITRATE": 500000,
"FD":true,
"DATA_BITRATE": 2000000,
"APP_NAME": "PyXCP-CANFD"
}"""
CONF_JSON = StringIO(JSON)
CONF_JSON.name = "conf_can_vector.json"
CONFIG = readConfiguration(CONF_JSON)

## コンフィグレーションを元にXCPマスターの生成

In [3]:
xm=Master('can',config=CONFIG)

## CONNECT

In [4]:
conn = xm.connect()
print(conn)

Container: 
    resource = Container: 
        dbg = False
        pgm = False
        stim = True
        daq = True
        calpag = False
    commModeBasic = Container: 
        optional = True
        slaveBlockMode = False
        addressGranularity = (enum) BYTE 0
        byteOrder = (enum) INTEL 0
    maxCto = 64
    maxDto = 64
    protocolLayerVersion = 16
    transportLayerVersion = 16


In [5]:
print("calpag =", xm.slaveProperties.supportsCalpag)
print("daq =", xm.slaveProperties.supportsDaq)
print("pgm =", xm.slaveProperties.supportsPgm)
print("stim =", xm.slaveProperties.supportsStim)

calpag = False
daq = True
pgm = False
stim = True


## GET_STATUS

In [6]:
status=xm.getStatus()
print(status)

Container: 
    sessionStatus = Container: 
        resume = False
        daqRunning = False
        clearDaqRequest = False
        storeDaqRequest = False
        storeCalRequest = False
    resourceProtectionStatus = Container: 
        dbg = False
        pgm = False
        stim = False
        daq = False
        calpag = False
    sessionConfiguration = 0


## SYNCH

In [7]:
res=xm.synch()
print(res)

bytearray(b'\x00')


## GET_COMM_MODE_INFO

In [8]:
CommModeInfo=xm.getCommModeInfo()
print(CommModeInfo)

Container: 
    commModeOptional = Container: 
        interleavedMode = False
        masterBlockMode = False
    maxBs = 99
    minSt = 0
    queueSize = 99
    xcpDriverVersionNumber = 16


## DOWNLOAD、UPLOADによるverify

In [9]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(31)
print(bhv.hex())

00000000000000000000000000000000000000000000000000000000000000


In [10]:
bhv = bytearray(bhv)
bhv[29] = 0xAA
bhv[30] = 0x55
xm.setMta(0x0000,0xff)
xm.download(bhv)

bytearray(b'')

In [11]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(31);
print(bhv.hex())

0000000000000000000000000000000000000000000000000000000000aa55


## DAQリストの構築

In [12]:
xm.freeDaq();

In [13]:
xm.allocDaq(2);

In [14]:
xm.allocOdt(0, 1);
xm.allocOdt(1, 1);

In [15]:
xm.allocOdtEntry(0, 0, 5);
xm.allocOdtEntry(1, 0, 1);

## ODT_ENTRYへの書き込み

In [16]:
DaqEntry = namedtuple("DaqEntry", "daq odt entry bitoff size ext addr")
de0 = (
    DaqEntry(daq=0, odt=0,  entry=0, bitoff=255, size=4, ext=0xff, addr=0x00000000),
    DaqEntry(daq=0, odt=0,  entry=1, bitoff=255, size=4, ext=0xff, addr=0x00000004),
    DaqEntry(daq=0, odt=0,  entry=2, bitoff=255, size=4, ext=0xff, addr=0x00000008),
    DaqEntry(daq=0, odt=0,  entry=3, bitoff=255, size=4, ext=0xff, addr=0x0000000C),
    DaqEntry(daq=0, odt=0,  entry=4, bitoff=255, size=4, ext=0xff, addr=0x00000010),
 
    DaqEntry(daq=1, odt=0,  entry=0, bitoff=255, size=4, ext=0xff, addr=0x00000004),
)


In [17]:
for daq, odt, entry, bitoff, size, ext, addr in de0:
    xm.setDaqPtr(daq, odt,  entry)
    xm.writeDaq(bitoff, size, ext, addr)

## DAQの起動

In [18]:
xm.setDaqListMode(0x10, 0, 1, 1, 0)
xm.setDaqListMode(0x02, 1, 3, 1, 0)

res = xm.startStopDaqList(0x02, 0)
print("startStopDaqList #0", res)
res = xm.startStopDaqList(0x02, 1)
print("startStopDaqList #1", res)

startStopDaqList #0 Container: 
    firstPid = 0
startStopDaqList #1 Container: 
    firstPid = 1


In [19]:
xm.startStopSynch(0x01);

## DAQの計測とSTIMによる外部からの値更新

In [20]:
stim_wait = 0.05
start = time.time()
stim_time =  time.time()+stim_wait-0.03

ram_update = [
    [0x11, 0x22, 0x33, 0x44],
    [0x55, 0x66, 0x77, 0x88],
    [0x99, 0xAA, 0xBB, 0xCC],
    [0x00, 0x00, 0x00, 0x00],
]
i = 0
str = ""
while True:
    queue_len = len(xm.transport.daqQueue)
    for _ in range(queue_len):
        daq = xm.transport.daqQueue.popleft()
        str += '%.6f, %s\n' % ( daq[3],daq[0].hex() )
    if time.time() > stim_time:
        bhv = bytearray([0x01]*5)
        bhv[1:5]=ram_update[i]
        xm.transport.send(bhv)
        str += 'stim %s\n' % bhv.hex()
        stim_time =  stim_time + stim_wait
        i = i+1
        if i > 3:
            i = 3
    if time.time() > start + 0.22:
        break
    time.sleep(0.010)

print(str)

1641457259.108966, 007602000000000000000000000000000000000000000000
1641457259.118985, 008002000000000000000000000000000000000000000000
1641457259.128897, 008a02000000000000000000000000000000000000000000
1641457259.138867, 009402000000000000000000000000000000000000000000
1641457259.148836, 009e02000000000000000000000000000000000000000000
1641457259.158831, 00a802000000000000000000000000000000000000000000
1641457259.168841, 00b202000000000000000000000000000000000000000000
stim 0111223344
1641457259.178762, 00bc02000000001122334400000000000000000000000000
1641457259.188772, 00c602000000001122334400000000000000000000000000
1641457259.198750, 00d002000000001122334400000000000000000000000000
1641457259.208720, 00da02000000001122334400000000000000000000000000
1641457259.218731, 00e402000000001122334400000000000000000000000000
stim 0155667788
1641457259.228667, 00ee02000000001122334400000000000000000000000000
1641457259.238662, 00f802000000005566778800000000000000000000000000
1641457259.24866

In [21]:
xm.startStopSynch(0x00);

In [22]:
xm.disconnect();